In [1]:
import os
import json
import random

INPUT_FOLDER = "final_dataset/"
OUTPUT_FILE = "pretrain_en.jsonl"

BUFFER_SIZE = 200_000   # controls shuffle quality vs memory use


def merge_and_shuffle():
    buffer = []
    total_written = 0
    file_count = 0

    # Remove old output if exists
    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)

    print("🔍 Scanning folder:", INPUT_FOLDER)

    jsonl_files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith(".jsonl")]
    jsonl_files.sort()  # optional

    print(f"📁 Found {len(jsonl_files)} JSONL files.\n")

    for filename in jsonl_files:
        path = os.path.join(INPUT_FOLDER, filename)
        file_count += 1

        print(f"📄 Processing file {file_count}/{len(jsonl_files)} → {filename}")

        file_entries = 0   # count entries for THIS file

        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    entry = json.loads(line)
                except json.JSONDecodeError:
                    continue  # skip invalid lines

                buffer.append(entry)
                file_entries += 1

                # flush buffer if too large
                if len(buffer) >= BUFFER_SIZE:
                    random.shuffle(buffer)
                    with open(OUTPUT_FILE, "a", encoding="utf-8") as out:
                        for item in buffer:
                            out.write(json.dumps(item, ensure_ascii=False) + "\n")
                    total_written += len(buffer)
                    print(f"   📝 Flushed {len(buffer)} entries (running total: {total_written})")
                    buffer = []

        print(f"   📌 Entries in {filename}: {file_entries}\n")

    # flush remaining buffer
    if buffer:
        random.shuffle(buffer)
        with open(OUTPUT_FILE, "a", encoding="utf-8") as out:
            for item in buffer:
                out.write(json.dumps(item, ensure_ascii=False) + "\n")
        total_written += len(buffer)

    print("🎉 MERGE COMPLETE!")
    print(f"📦 Total entries written: {total_written}")
    print(f"📄 Output saved to: {OUTPUT_FILE}")

    return total_written


if __name__ == "__main__":
    merge_and_shuffle()

🔍 Scanning folder: final_dataset/
📁 Found 6 JSONL files.

📄 Processing file 1/6 → pretrain_alpaca_gpt4_en.jsonl
   📌 Entries in pretrain_alpaca_gpt4_en.jsonl: 23093

📄 Processing file 2/6 → pretrain_c4.jsonl
   📝 Flushed 200000 entries (running total: 200000)
   📝 Flushed 200000 entries (running total: 400000)
   📌 Entries in pretrain_c4.jsonl: 500000

📄 Processing file 3/6 → pretrain_code_alpaca20k.jsonl
   📌 Entries in pretrain_code_alpaca20k.jsonl: 16746

📄 Processing file 4/6 → pretrain_smollm.jsonl
   📝 Flushed 200000 entries (running total: 600000)
   📝 Flushed 200000 entries (running total: 800000)
   📝 Flushed 200000 entries (running total: 1000000)
   📝 Flushed 200000 entries (running total: 1200000)
   📝 Flushed 200000 entries (running total: 1400000)
   📌 Entries in pretrain_smollm.jsonl: 999978

📄 Processing file 5/6 → pretrain_stack_smol_code.jsonl
   📌 Entries in pretrain_stack_smol_code.jsonl: 1428

📄 Processing file 6/6 → pretrain_wikipedia_en.jsonl
   📝 Flushed 200000 